# Descripción del problema
Una persona puede invertir en 7 distintas inversiones. Cada una de ellas le ofrece una ganancia a un plazo de un año, pero requiere de un monto fijo a invertir, los detalles se muestran enseguida:

|          | Inversión 1 | Inversión 2 | Inversión 3 | Inversión 4 | Inversión 5 | Inversión 6 | Inversión 7 |
|---|---|---|---|---|---|---|---|
|Ganancia (en miles de pesos) | 12| 8 | 17 | 11 |6 |2 | 2|
|Monto a invertir (en miles de pesos) |4|3|7|5|3|2|3|

Si la persona dispone únicamente de un capital de $17,000, ¿cuáles inversiones debe elegir para maximizar sus ganancias?

## 1. Realice la modelación del problema (8 pts)
En cada una de las siguientes celdas, agregue una descripción correspondiente para explicar el contenido y la ecuación matemática que represente la parte del modelo en cuestión.

### Conjuntos
$I$: Conjunto de las posibles inversiones que podemos realizar: {Inversión 1. Inversión 2, Inversión 3, Inversión 4, Inversión 5, Inversión 6, Inversión 7}. Representaremos respectivamente a $I$ como $$I=\{1, 2, 3, 4, 5, 6, 7\}$$ donde $i\in I$ representa a uno de los elementos en $I$.

### Parámetros
Los parámetros considerados en el presente modelo son:
* $P$: La cantidad disponible en miles de pesos para hacer las inversiones. $$P=17$$

* $G_i$: La ganancia en miles de pesos que implica hacer la inversión $i\in I$.

* $M_i$: El monto inicial necesario para poder realizar la inversión $i\in I$.

Estos parámetros están presentes en la descripción del problema.

### Variables de decisión
La variable de decisión utilizada para este modelo será
\begin{equation}c_i = \left \{ \begin{matrix} 1 & \text{si se elige la inversión }i
\\ 0 & \text{si no}\end{matrix}\right.\end{equation}

### Función objetivo
Maximizar ganancia en inversiones:
$$\text{max}\sum_{i=1}^{7} c_i \cdot G_i$$

### Restricciones
* La del monto disponible en miles de pesos para realizar las inversiones: $$\sum_{i=1}^{7}c_i \cdot M_i \leq P$$

## 2. Utilice la librería `pulp` para determinar la solución del modelo (4pts)
En cada una de las siguientes celdas agregue el código indicado en el comentario

### Cargar los archivos de Google Drive e instalar biblioteca `pulp`
Primero importaremos los documentos fuente a Google Colab desde la carpeta de Google Drive.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


Ahora verificaremos que podemos acceder a nuestros documentos.

In [ ]:
!ls "gdrive/My Drive/Colab Notebooks/Investigación de Operaciones/Examen_1"

ProblemaDeInversion.csv


Una vez verificados los documentos en nuestra carpeta, cargaremos la biblioteca `pandas` y nuestro archivo `ProblemaDeInversion.csv` que contiene los datos de la tabla de arriba.

In [ ]:
import pandas as pd
ProbInversion = pd.read_csv("gdrive/My Drive/Colab Notebooks/Investigación de Operaciones/Examen_1/ProblemaDeInversion.csv")

Para verificar que los archivos fueron cargados correctamente, los mostramos dentro de nuestra Notebook de Python.

In [ ]:
ProbInversion

,Ganancia,Inversion_Inicial
0,12,4
1,8,3
2,17,7
3,11,5
4,6,3
5,2,2
6,2,3


Una vez cargados los datos, procederemos a instalar la biblioteca `pulp` en nuestra sesión de Colab. Importaremos las funciones `LpProblem`, `LpVariable`, `LpMaximize`, `LpStatus` y `lpSum`.

In [ ]:
!pip install pulp
from pulp import LpProblem, LpVariable, LpMaximize, LpStatus, lpSum

     |████████████████████████████████| 40.6MB 95kB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for amply: filename=amply-0.1.2-cp36-none-any.whl size=16573 sha256=ea62fa4d27c91387f2a6a56cfdc012801975b414f0528a6baaea5ce1e5fe72b2
  Stored in directory: /root/.cache/pip/wheels/84/18/f7/e5c3ed13ed5bb721763f77d4a924331d59ef115ce61c9d26eb
Successfully built amply


### Declarar el modelo y añadir todos sus parámetros y restricciones
Ahora procederemos a **crear nuestro modelo principal**, el de maximizar la ganancia de nuestras inversiones. Como vamos a maximizar, al parámetro `sense` lo pondremos como `sense=LpMaximize`.

In [ ]:
# Escribe aquí la declaración del modelo
modelo = LpProblem("Modelo_Inversiones", sense=LpMaximize)

Una vez creado nuestro modelo, necesitaremos **declarar nuestros parámetros** $G_i$ y $M_i$ y tomar sus datos del documento importado y añadirlo a nuestro modelo. Recordemos que `i` representa alguna inversión $i∈I$.

In [ ]:
# Escribe aquí los parámetros del modelo
var_G = {i: LpVariable(name=f"G_{i+1}") for i in range(7)}
var_M = {i: LpVariable(name=f"M_{i+1}") for i in range(7)}
for i in range(7):
  var_G[i] = ProbInversion.Ganancia.iloc[i]
  var_M[i] = ProbInversion.Inversion_Inicial.iloc[i]

Declaramos nuestra **variable de decisión** $c_i$, donde `i` representa a alguna inversión $i \in I$.

In [ ]:
# Escribe aquí las variables de decisión
var_c = {i: LpVariable(name=f"c_{i+1}", cat='Binary') for i in range(7)}

Una vez declaradas las variables y parámetros, añadiremos a nuestro modelo **la restricción** que definimos anteriormente.

In [ ]:
# Escribe aquí las restricciones
modelo += lpSum(var_c[i] * var_M[i] for i in range(7)) <= 17

Ahora procederemos a añadir en nuestro modelo la **función objetivo** a la que queremos encontrarle solución y lo resolvemos utilizando la función `.solve()`. Guardamos nuestro modelo en la variable `solucion`. Verificamos que la solución sea óptima con la función `LpStatus`.

In [ ]:
# Escribe aquí la función objetivo
modelo += lpSum(var_c[i] * var_G[i] for i in range(7))
solucion = modelo.solve()
LpStatus[solucion]

'Optimal'

Imprimimos los resultados de nuestro modelo y las inversiones que se necesitan hacer para lograr maximizar las ganancias.

In [ ]:
#Mayor ganancia al invertir los $ 17,000 de acuerdo con nuestro modelo
inversion = 0
for i in range(7):
  inversion = var_c[i].value() * var_M[i] + inversion
print(f"La ganancia al haber hecho las inversiones es de:  $", modelo.objective.value() * 1000)
print(f"Se realizó una inversión inicial de:  $", inversion * 1000)
print("Las inversiones que se necesitan hacer para lograr la ganancia anterior son las siguientes:")
for i in range(7):
  if(var_c[i].value() == 1):
    print(f"   - Inversión #{i+1}")

La ganancia al haber hecho las inversiones es de:  $ 43000.0
Se realizó una inversión inicial de:  $ 17000.0
Las inversiones que se necesitan hacer para lograr la ganancia anterior son las siguientes:
   - Inversión #1
   - Inversión #2
   - Inversión #3
   - Inversión #5


### Resultados
Escribe los resultados obtenidos de las variables de decisión y la función objetivo. ¿Qué recomendación se le daría al inversionista?

De acuerdo al modelo anterior, se le recomienda al inversionista hacer las inversiones #1, #2, #3 y #5, donde hará uso del total de su capital inicial (\$ 17,000.00) y logrará una ganancia de \$ 43,000.00, aumentando en \$ 26,000.00 su capital.